<a href="https://colab.research.google.com/github/samiravaez/IMDB-Scraper/blob/main/imdb_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from requests import get
from bs4 import BeautifulSoup
from warnings import warn
from time import sleep
from random import randint
import numpy as np, pandas as pd
import seaborn as sns
import re
import csv

pages = np.arange(501, 2000, 50)


# initialize the data structure to store the scraped content
titles = []
years = []
genres = []
runtimes = []
imdb_ratings = []
imdb_ratings_standardized = []
metascores = []
votes = []
descriptions = []
counter = 0
for page in pages:
  try:
    response = get ("https://www.imdb.com/search/title/?title_type=feature&year=2022-01-01,2022-12-31&start="
    +str(page)+"&ref_=adv_nxt")
    response.raise_for_status()
    sleep(randint(10,15))

  except requests.exceptions.RequestException as err:
    print('Request failed as:',err)

  page_imdb = BeautifulSoup(response.text,'html.parser')
  movie_containers = page_imdb.find_all('div', class_ = 'lister-item mode-advanced')

  for movieInf in movie_containers:


    #title
    find_title = movieInf.h3
    title = None
    if find_title:
      title = find_title.a.text if find_title.a else None
    titles.append(title)


    #year released
    year = None
    if movieInf.h3.find('span', class_= 'lister-item-year text-muted unbold'):
      year = movieInf.h3.find('span', class_= 'lister-item-year text-muted unbold').text
      pattern = r'^\(([^0-9]+)\)'
      match_movie_version = re.match(pattern, year)

      if match_movie_version:
        movie_version = match_movie_version.group(1)
        titles[counter] += '/'+ movie_version

      year = re.sub(pattern+'|[()]', '', year)
      if not year:
       year = None
    years.append(year)

    ##IMDB rating
    find_div = movieInf.find('div', class_= 'ratings-bar')
    rating = None
    if find_div:
      strong_tag = find_div.strong
      rating = strong_tag.text if strong_tag else None
    imdb_ratings.append(rating)


    #genre
    genre = None
    find_genre = movieInf.p.find('span', class_ = 'genre')
    if find_genre:
      genre = find_genre.text.replace("\n", "").replace(',','|').replace(' ','') # remove the whitespace character, strip, and split to create an array of genres
    genres.append(genre)


    #runtime
    find_time = movieInf.p.find('span', class_ = 'runtime')
    runtime = None
    if find_time:
      runtime = find_time.text.replace(" min", "")
    runtimes.append(runtime)


    #Metascore
    meta_find = movieInf.find('div', class_ = 'inline-block ratings-metascore')
    meta_score = None
    if meta_find:
      meta_score = meta_find.span.text.replace(" ","") if meta_find.span else None
    metascores.append(meta_score)


    #Number of votes
    movie_find = movieInf.find('span', attrs = {'name':'nv'})
    vote = None
    if movie_find:
      find_data = movie_find['data-value']
      vote =  find_data if find_data else None
    votes.append(vote)


    # descriptions
    find_description = movieInf.find('div', class_ = 'lister-item-content')
    descrip = None
    if find_description:
      find_descrip = find_description.find_all('p', class_='text-muted')
      descrip = find_descrip[1].text if find_descrip else None
    descriptions.append(descrip)


    counter += 1

movies2022 = pd.DataFrame({'movie': titles,
                      'year': years,
                      'rating': imdb_ratings,
                      'genre': genres,
                      'runtime': runtimes,
                      'imdb_rating': imdb_ratings,
                      'metascore': metascores,
                      'votes': votes,
                      'descriptions':descriptions}
                      )

# final_df = movies2022.loc[movies2022['year'] != 'ovie']
# movies2022.loc[:, 'year'] = pd.to_numeric(movies2022['year'])
# print(movies2022.iloc[2,:])
movies2022['descriptions'] = movies2022['descriptions'].replace(r"\n", "", regex=True)
movies2022.to_csv('movies2022.csv', index=False)
